In [1]:
import numpy as np
import os
import shutil
import subprocess



def write_list_to_file(file_path, my_list):
    try:
        # Open the file in write mode ('w')
        with open(file_path, 'w') as file:
            # Loop through each element in the list
            for item in my_list:
                # Write the item followed by a newline character to separate items
                file.write(str(item) + '\n')
        print("List has been successfully written to the file.")
    except IOError:
        print("An error occurred while writing the file.")



nregions = 3

latency_map = np.zeros((nregions, nregions))

latency_map[0,1] = 65
latency_map[1,0] = 65


latency_map[0,2] = 112
latency_map[2,0] = 112

latency_map[2,1] = 70
latency_map[1,2] = 70


# 2 cluster map

node_cluster_map = {
    0:0,
    1:0,
    2:0,
    3:0,
    4:1,
    5:1,
    6:1,
    7:1,
    8:2,
    9:2,
    10:2,
    11:2,
    12:3,
    13:3,
    14:3,
    15:3,
    16:4,
    17:4,
    18:4,
    19:4,
    20:5,
    21:5,
    22:5,
    23:5,
    24:6,
    25:6,
    26:6,
    27:6,
    28:7,
    29:7,
    30:7,
    31:7,
    32:8,
    33:8,
    34:8,
    35:8,
    36:9,
    37:9,
    38:9,
    39:9,
    40:10,
    41:10,
    42:10,
    43:10,
    44:11,
    45:11,
    46:11,
    47:11,
    48:0,
    49:1,
    50:2,
    51:3,
    52:4,
    53:5,
    54:6,
    55:7,
    56:8,
    57:9,
    58:10,
    59:11,
    
}


with open('all_internal_ips') as f:
    ipList = f.read().splitlines()
ipList = [x for x in ipList if x!='None']

ipList, len(ipList)



os.system('cd simul_latency/; rm *')

for i, ip in enumerate(ipList):
    print(i,ip)
    
    Lines = []
    
    Lines.append('tc qdisc add dev eth0 root handle 1: htb')
    Lines.append('')
    
    
    count_ip = 1
    
    for  j, ip_loop in enumerate(ipList):
        if j!=i:
            
            cls_i = node_cluster_map[i]%nregions
            cls_j = node_cluster_map[j]%nregions
            
            latency_delay = latency_map[cls_i, cls_j]
            
            Lines.append('tc class add dev eth0 parent 1: classid 1:'+str(count_ip)+' htb rate 1000mbit')
            Lines.append('tc filter add dev eth0 parent 1: protocol ip prio '+str(count_ip)+' u32 flowid 1:'+str(count_ip)+' match ip dst '+ip_loop+'/32')
            Lines.append('tc qdisc add dev eth0 parent 1:'+str(count_ip)+' handle '+str(count_ip)+'0: netem delay '+str(latency_delay)+'ms')
            Lines.append('')
            
            
            count_ip = count_ip + 1

    
    
    write_list_to_file('simul_latency/node_'+str(i)+'.sh', Lines)
    
    
#     break




